<a href="https://colab.research.google.com/github/Kolexx/Kolexx-FPL-Team-Composition-Performance-Analysis-Data-Extraction-Power-BI-Visualization/blob/main/scripts/FPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time

In [ ]:
team_id = '4013862'
history_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/history/"

history_data = requests.get(history_url).json()
history_df = pd.DataFrame(history_data['current'])
history_df.head()

,event,points,total_points,rank,rank_sort,overall_rank,percentile_rank,bank,value,event_transfers,event_transfers_cost,points_on_bench
0,1,42,42,7576260,7635956,7576260,90,0,1000,0,0,0
1,2,95,133,498769,501406,3520163,10,0,1000,2,4,0
2,3,82,215,1107888,1134674,1977827,15,0,1003,0,0,0
3,4,62,273,2148159,2153725,1575231,25,9,1007,3,4,1
4,5,92,361,52665,53516,375837,1,4,1013,2,4,8


In [ ]:
START_GAMEWEEK = 1
END_GAMEWEEK = 26

# Fetch all player data from the FPL API
players_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
players_data = requests.get(players_url).json()
players_df = pd.DataFrame(players_data['elements'])

# Select relevant columns from players data
players_df = players_df[['id', 'web_name']]
players_df.rename(columns={'id': 'element', 'web_name': 'playerName'}, inplace=True)

# Initialize an empty list to store all gameweek data
all_picks = []

for gameweek in range(START_GAMEWEEK, END_GAMEWEEK + 1):
    picks_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gameweek}/picks/"
    response = requests.get(picks_url)

    if response.status_code == 200:
        picks_data = response.json()
        picks = picks_data.get('picks', [])

        for pick in picks:
            pick['gameweek'] = gameweek  # Add gameweek number to each row
            all_picks.append(pick)
    else:
        print(f"Failed to fetch data for Gameweek {gameweek}")

# Convert the list into a DataFrame
picks_df = pd.DataFrame(all_picks)

# Merge picks_df with players_df to add player names
picks_df = picks_df.merge(players_df, on='element', how='left')

# Display the first few rows
picks_df


,element,position,multiplier,is_captain,is_vice_captain,element_type,gameweek,playerName
0,235,1,1,False,True,1,1,Pickford
1,326,2,1,False,False,2,1,Konaté
2,171,3,1,False,False,2,1,Gusto
3,505,4,1,False,False,2,1,Udogie
4,255,5,1,False,False,2,1,Robinson
...,...,...,...,...,...,...,...,...
385,447,11,1,False,False,4,26,Wood
386,396,12,0,False,False,1,26,Dúbravka
387,159,13,0,False,False,2,26,Chalobah
388,252,14,0,False,False,4,26,Raúl


In [ ]:
# Convert to DataFrame
players_df = pd.DataFrame(players_data['elements'])
players_df = players_df[['id', 'web_name', 'element_type', 'now_cost', 'team']]
players_df.rename(columns={'id': 'element'}, inplace=True)

# Fetch team data
teams_df = pd.DataFrame(players_data['teams'])[['id', 'name']]
teams_df.rename(columns={'id': 'team', 'name': 'team_name'}, inplace=True)

# Fetch picks for all gameweeks
all_picks = []

for gameweek in range(START_GAMEWEEK, END_GAMEWEEK + 1):
    picks_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gameweek}/picks/"
    response = requests.get(picks_url)

    if response.status_code == 200:
        picks_data = response.json()
        picks = picks_data.get('picks', [])

        for pick in picks:
            pick['gameweek'] = gameweek  # Add gameweek number
            all_picks.append(pick)
    else:
        print(f"Failed to fetch data for Gameweek {gameweek}")

# Convert the list into a DataFrame
picks_df = pd.DataFrame(all_picks)

# Extract unique player IDs from your team
unique_player_ids = picks_df['element'].unique()

# Filter players_df to include only your team’s players
my_team_players_df = players_df[players_df['element'].isin(unique_player_ids)]

# Merge to get team names
my_team_players_df = my_team_players_df.merge(teams_df, on='team', how='left')

# Select relevant columns including team_name
my_team_players_df = my_team_players_df[['element', 'web_name', 'element_type', 'now_cost', 'team_name']]

# Display the first few rows
my_team_players_df


,element,web_name,element_type,now_cost,team_name
0,3,Gabriel,2,63,Arsenal
1,99,Mbeumo,3,80,Brentford
2,129,João Pedro,4,54,Brighton
3,148,Welbeck,4,55,Brighton
4,154,Bergström,1,39,Chelsea
5,159,Chalobah,2,44,Chelsea
6,171,Gusto,2,49,Chelsea
7,182,Palmer,3,111,Chelsea
8,185,Sánchez,1,46,Chelsea
9,560,Neto,3,62,Chelsea


In [ ]:
# Fetch player data
players_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(players_url)

if response.status_code != 200:
    raise Exception(f"Error fetching player data: {response.status_code}")

players_data_json = response.json()  # Correct variable name

if "elements" not in players_data_json:
    raise KeyError("Missing 'elements' in players_data_json")  # Extra check

# Correctly map player ID to name
players_data = {player['id']: player['web_name'] for player in players_data_json['elements']}

# Define the gameweeks to check (from GW1 to the latest GW)
gameweeks = list(range(1, 27))
team_id = 4013862  # Replace with your actual FPL team ID

# Create a list to store the data
data = []

for gw in gameweeks:
    # Get your team's picks for the gameweek
    picks_url = f'https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/'
    picks_response = requests.get(picks_url)

    if picks_response.status_code != 200:
        print(f"Skipping GW{gw}: Picks data unavailable")
        continue

    picks_data = picks_response.json()

    # Get live player data for the gameweek (to fetch points)
    live_url = f"https://fantasy.premierleague.com/api/event/{gw}/live/"
    live_response = requests.get(live_url)

    if live_response.status_code != 200:
        print(f"Skipping GW{gw}: Live data unavailable")
        continue

    live_data = live_response.json()
    live_stats = {player['id']: player['stats']['total_points'] for player in live_data['elements']}

    for player in picks_data['picks']:
        player_id = player['element']
        player_name = players_data.get(player_id, "Unknown Player")
        player_points = live_stats.get(player_id, 0)
        total_points = player_points * player['multiplier']

        # Check if the player is on the bench (Positions 12-15)
        on_bench = player['position'] >= 12

        # Store data in a list
        data.append([gw, player_name, player_id, player_points, player['multiplier'], total_points, on_bench])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Gameweek", "Player", "Player ID", "Base Points", "Multiplier", "Total Points", "On Bench"])

df


,Gameweek,Player,Player ID,Base Points,Multiplier,Total Points,On Bench
0,1,Pickford,235,1,1,1,False
1,1,Konaté,326,1,1,1,False
2,1,Gusto,171,1,1,1,False
3,1,Udogie,505,2,1,2,False
4,1,Robinson,255,2,1,2,False
...,...,...,...,...,...,...,...
385,26,Wood,447,5,1,5,False
386,26,Dúbravka,396,0,0,0,True
387,26,Chalobah,159,1,0,0,True
388,26,Raúl,252,2,0,0,True


In [ ]:
distinct_players = df["Player ID"].nunique()
print(f"Total distinct players ever owned: {distinct_players}")


Total distinct players ever owned: 36


In [ ]:
# Fetch player data
players_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(players_url)

if response.status_code != 200:
    raise Exception(f"Error fetching player data: {response.status_code}")

players_data_json = response.json()

if "elements" not in players_data_json:
    raise KeyError("Missing 'elements' in players_data_json")

# Map player ID to name and initial prices
players_data = {player['id']: player['web_name'] for player in players_data_json['elements']}
initial_prices = {player['id']: player['now_cost'] / 10 for player in players_data_json['elements']}  # Ensure this exists

# Fetch transfer history to track bought and sold players
transfers_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/transfers/"
transfers_response = requests.get(transfers_url)

if transfers_response.status_code == 404:
    transfers_data = []
else:
    transfers_data = transfers_response.json()

# Track players sold and their selling price
sold_players = {transfer["element_out"]: transfer["element_out_cost"] / 10 for transfer in transfers_data}

# Fetch initial team data (before Gameweek 1)
initial_team_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/1/picks/"
initial_team_response = requests.get(initial_team_url)

if initial_team_response.status_code == 404:
    initial_team_data = []
else:
    initial_team_data = initial_team_response.json().get('picks', [])

# Store players with their buying and selling prices
players_info = []

# Add players from the initial team selection (before GW1)
for player in initial_team_data:
    player_id = player['element']
    buying_price = initial_prices.get(player_id, 0)
    selling_price = sold_players.get(player_id, float('nan'))  # Set to NaN if not sold
    players_info.append((players_data.get(player_id, "Unknown Player"), player_id, buying_price, selling_price, 0))  # 0 for Game Week

# Include any transfers made during the season
for transfer in transfers_data:
    gw = transfer["event"]
    buying_price = transfer.get("element_in_cost", 0) / 10  # Buying price from transfer data
    selling_price = sold_players.get(transfer["element_in"], float('nan'))  # Get selling price if sold
    players_info.append((players_data.get(transfer["element_in"], "Unknown Player"), transfer["element_in"], buying_price, selling_price, gw))

# Create a DataFrame from the players' information
players_df = pd.DataFrame(players_info, columns=["Player", "Player ID", "Buying Price", "Selling Price", "Game Week"])

# Drop duplicates to ensure each player appears only once
df2 = players_df.drop_duplicates(subset=["Player ID"])

# Display the final DataFrame
df2


,Player,Player ID,Buying Price,Selling Price,Game Week
0,Pickford,235,5.1,4.9,0
1,Konaté,326,5.2,5.1,0
2,Gusto,171,4.9,5.0,0
3,Udogie,505,4.8,4.9,0
4,Robinson,255,5.1,NaN,0
5,Bowen,514,7.3,7.5,0
6,B.Fernandes,366,8.3,8.4,0
7,Palmer,182,11.1,NaN,0
8,M.Salah,328,13.7,NaN,0
9,Haaland,351,14.7,14.8,0


In [ ]:
# API endpoint for gameweek history
url = f'https://fantasy.premierleague.com/api/entry/{team_id}/history/'

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an error if request fails
    gameweek_data = response.json()
except requests.exceptions.RequestException as e:
    print("Error fetching data:", e)
    exit()

# Extract overall rank for each Gameweek
gameweeks = []
for gameweek in gameweek_data.get('current', []):  # Use .get() to prevent KeyErrors
    gameweeks.append({
        "Gameweek": gameweek['event'],
        "Overall Rank": gameweek['overall_rank']
    })

# Convert to a Pandas DataFrame
history_df = pd.DataFrame(gameweeks)
history_df

,Gameweek,Overall Rank
0,1,7576260
1,2,3520163
2,3,1977827
3,4,1575231
4,5,375837
5,6,226169
6,7,365011
7,8,397136
8,9,657378
9,10,761389


In [ ]:
# Rename 'element' to 'Player ID' in necessary DataFrames
picks_df.rename(columns={"element": "Player ID"}, inplace=True)
my_team_players_df.rename(columns={"element": "Player ID"}, inplace=True)
df.rename(columns={"element": "Player ID"}, inplace=True)
df2.rename(columns={"element": "Player ID"})
picks_df


,Player ID,position,multiplier,is_captain,is_vice_captain,element_type,gameweek
0,235,1,1,False,True,1,1
1,326,2,1,False,False,2,1
2,171,3,1,False,False,2,1
3,505,4,1,False,False,2,1
4,255,5,1,False,False,2,1
...,...,...,...,...,...,...,...
385,447,11,1,False,False,4,26
386,396,12,0,False,False,1,26
387,159,13,0,False,False,2,26
388,252,14,0,False,False,4,26


In [ ]:
my_team_players_df


,Player ID,web_name,element_type,now_cost,team_name
0,3,Gabriel,2,63,Arsenal
1,99,Mbeumo,3,80,Brentford
2,129,João Pedro,4,54,Brighton
3,148,Welbeck,4,55,Brighton
4,154,Bergström,1,39,Chelsea
5,159,Chalobah,2,44,Chelsea
6,171,Gusto,2,49,Chelsea
7,182,Palmer,3,111,Chelsea
8,185,Sánchez,1,46,Chelsea
9,560,Neto,3,62,Chelsea


In [ ]:
df


,Gameweek,Player,Player ID,Base Points,Multiplier,Total Points,On Bench
0,1,Pickford,235,1,1,1,False
1,1,Konaté,326,1,1,1,False
2,1,Gusto,171,1,1,1,False
3,1,Udogie,505,2,1,2,False
4,1,Robinson,255,2,1,2,False
...,...,...,...,...,...,...,...
385,26,Wood,447,5,1,5,False
386,26,Dúbravka,396,0,0,0,True
387,26,Chalobah,159,1,0,0,True
388,26,Raúl,252,2,0,0,True


In [ ]:
df2

,Player,Player ID,Buying Price,Selling Price,Game Week
0,Pickford,235,5.1,4.9,0
1,Konaté,326,5.2,5.1,0
2,Gusto,171,4.9,5.0,0
3,Udogie,505,4.8,4.9,0
4,Robinson,255,5.1,NaN,0
5,Bowen,514,7.3,7.5,0
6,B.Fernandes,366,8.3,8.4,0
7,Palmer,182,11.1,NaN,0
8,M.Salah,328,13.7,NaN,0
9,Haaland,351,14.7,14.8,0


In [ ]:
# Merge picks_df with my_team_players_df to get position info
merged_df = picks_df.merge(my_team_players_df, on="Player ID", how="left")

merged_df


,Player ID,position,multiplier,is_captain,is_vice_captain,element_type_x,gameweek,web_name,element_type_y,now_cost,team_name
0,235,1,1,False,True,1,1,Pickford,1,51,Everton
1,326,2,1,False,False,2,1,Konaté,2,52,Liverpool
2,171,3,1,False,False,2,1,Gusto,2,49,Chelsea
3,505,4,1,False,False,2,1,Udogie,2,48,Spurs
4,255,5,1,False,False,2,1,Robinson,2,51,Fulham
...,...,...,...,...,...,...,...,...,...,...,...
385,447,11,1,False,False,4,26,Wood,4,72,Nott'm Forest
386,396,12,0,False,False,1,26,Dúbravka,1,43,Newcastle
387,159,13,0,False,False,2,26,Chalobah,2,44,Chelsea
388,252,14,0,False,False,4,26,Raúl,4,56,Fulham


In [ ]:
if "Gameweek" in merged_df.columns and "Player ID" in merged_df.columns:
    if "Gameweek" in df.columns and "Player ID" in df.columns:
        merged_df = merged_df.merge(df, on=["Gameweek", "Player ID"], how="left")
    else:
        print("Error: 'Gameweek' or 'Player ID' missing from df")
else:
    print("Error: 'Gameweek' or 'Player ID' missing from merged_df")


Error: 'Gameweek' or 'Player ID' missing from merged_df


In [ ]:
print(merged_df.columns)


Index(['Player ID', 'position', 'multiplier', 'is_captain', 'is_vice_captain',
       'element_type_x', 'gameweek', 'web_name', 'element_type_y', 'now_cost',
       'team_name'],
      dtype='object')


In [ ]:
merged_df.rename(columns={"gameweek": "Gameweek"}, inplace=True)


In [ ]:
merged_df = merged_df.merge(df, on=["Gameweek", "Player ID"], how="left")


In [ ]:
merged_df

,Player ID,position,multiplier,is_captain,is_vice_captain,element_type_x,Gameweek,web_name,element_type_y,now_cost,team_name,Player,Base Points,Multiplier,Total Points,On Bench
0,235,1,1,False,True,1,1,Pickford,1,51,Everton,Pickford,1,1,1,False
1,326,2,1,False,False,2,1,Konaté,2,52,Liverpool,Konaté,1,1,1,False
2,171,3,1,False,False,2,1,Gusto,2,49,Chelsea,Gusto,1,1,1,False
3,505,4,1,False,False,2,1,Udogie,2,48,Spurs,Udogie,2,1,2,False
4,255,5,1,False,False,2,1,Robinson,2,51,Fulham,Robinson,2,1,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,447,11,1,False,False,4,26,Wood,4,72,Nott'm Forest,Wood,5,1,5,False
386,396,12,0,False,False,1,26,Dúbravka,1,43,Newcastle,Dúbravka,0,0,0,True
387,159,13,0,False,False,2,26,Chalobah,2,44,Chelsea,Chalobah,1,0,0,True
388,252,14,0,False,False,4,26,Raúl,4,56,Fulham,Raúl,2,0,0,True


In [ ]:
# Merge with df2 to include price changes
merged_df2 = pd.merge(merged_df, df2, on="Player ID", how="left")
merged_df2

,Player ID,position,multiplier,is_captain,is_vice_captain,element_type_x,Gameweek,web_name,element_type_y,now_cost,team_name,Player_x,Base Points,Multiplier,Total Points,On Bench,Player_y,Buying Price,Selling Price,Game Week
0,235,1,1,False,True,1,1,Pickford,1,51,Everton,Pickford,1,1,1,False,Pickford,5.1,4.9,0
1,326,2,1,False,False,2,1,Konaté,2,52,Liverpool,Konaté,1,1,1,False,Konaté,5.2,5.1,0
2,171,3,1,False,False,2,1,Gusto,2,49,Chelsea,Gusto,1,1,1,False,Gusto,4.9,5.0,0
3,505,4,1,False,False,2,1,Udogie,2,48,Spurs,Udogie,2,1,2,False,Udogie,4.8,4.9,0
4,255,5,1,False,False,2,1,Robinson,2,51,Fulham,Robinson,2,1,2,False,Robinson,5.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,447,11,1,False,False,4,26,Wood,4,72,Nott'm Forest,Wood,5,1,5,False,Wood,7.0,NaN,25
386,396,12,0,False,False,1,26,Dúbravka,1,43,Newcastle,Dúbravka,0,0,0,True,Dúbravka,4.2,NaN,21
387,159,13,0,False,False,2,26,Chalobah,2,44,Chelsea,Chalobah,1,0,0,True,Chalobah,4.4,NaN,10
388,252,14,0,False,False,4,26,Raúl,4,56,Fulham,Raúl,2,0,0,True,Raúl,5.4,NaN,5


In [ ]:
# Ensure "Buying Price" exists in merged_df2
if "Buying Price" in merged_df2.columns:
    nan_buying_price = merged_df2[merged_df2["Buying Price"].isna()].drop_duplicates(subset=["Player ID"])
else:
    print("Column 'Buying Price' not found in merged_df2!")


In [ ]:
# Ensure "Buying Price" exists in merged_df2
if "Buying Price" in merged_df2.columns:
    not_nan_buying_price = merged_df2[merged_df2["Buying Price"].notna()].drop_duplicates(subset=["Player ID"])
else:
    print("Column 'Buying Price' not found in merged_df2!")
not_nan_buying_price

,Player ID,position,multiplier,is_captain,is_vice_captain,element_type_x,Gameweek,web_name,element_type_y,now_cost,team_name,Player_x,Base Points,Multiplier,Total Points,On Bench,Player_y,Buying Price,Selling Price,Game Week
0,235,1,1,False,True,1,1,Pickford,1,51,Everton,Pickford,1,1,1,False,Pickford,5.1,4.9,0
1,326,2,1,False,False,2,1,Konaté,2,52,Liverpool,Konaté,1,1,1,False,Konaté,5.2,5.1,0
2,171,3,1,False,False,2,1,Gusto,2,49,Chelsea,Gusto,1,1,1,False,Gusto,4.9,5.0,0
3,505,4,1,False,False,2,1,Udogie,2,48,Spurs,Udogie,2,1,2,False,Udogie,4.8,4.9,0
4,255,5,1,False,False,2,1,Robinson,2,51,Fulham,Robinson,2,1,2,False,Robinson,5.1,NaN,0
5,514,6,1,False,False,3,1,Bowen,3,73,West Ham,Bowen,2,1,2,False,Bowen,7.3,7.5,0
6,366,7,1,False,False,3,1,B.Fernandes,3,83,Man Utd,B.Fernandes,3,1,3,False,B.Fernandes,8.3,8.4,0
7,182,8,1,False,False,3,1,Palmer,3,111,Chelsea,Palmer,2,1,2,False,Palmer,11.1,NaN,0
8,328,9,1,False,False,3,1,M.Salah,3,137,Liverpool,M.Salah,14,1,14,False,M.Salah,13.7,NaN,0
9,351,10,2,True,False,4,1,Haaland,4,147,Man City,Haaland,7,2,14,False,Haaland,14.7,14.8,0


In [ ]:
distinct_players1 = nan_buying_price["Player ID"].nunique()
print(f"Total distinct players ever owned: {distinct_players1}")
distinct_players2 = not_nan_buying_price["Player ID"].nunique()
print(f"Total distinct players ever owned: {distinct_players2}")

Total distinct players ever owned: 0
Total distinct players ever owned: 36


In [ ]:
# Merge merged_df2 with history_df on "Gameweek"
final_df = merged_df2.merge(history_df, on="Gameweek", how="left")

# Display final DataFrame
final_df

,Player ID,position,multiplier,is_captain,is_vice_captain,element_type_x,Gameweek,web_name,element_type_y,now_cost,team_name,Player_x,Base Points,Multiplier,Total Points,On Bench,Player_y,Buying Price,Selling Price,Game Week,Overall Rank
0,235,1,1,False,True,1,1,Pickford,1,51,Everton,Pickford,1,1,1,False,Pickford,5.10,4.90,0,7576260
1,326,2,1,False,False,2,1,Konaté,2,52,Liverpool,Konaté,1,1,1,False,Konaté,5.20,5.10,0,7576260
2,171,3,1,False,False,2,1,Gusto,2,49,Chelsea,Gusto,1,1,1,False,Gusto,4.90,5.00,0,7576260
3,505,4,1,False,False,2,1,Udogie,2,48,Spurs,Udogie,2,1,2,False,Udogie,4.80,4.90,0,7576260
4,255,5,1,False,False,2,1,Robinson,2,51,Fulham,Robinson,2,1,2,False,Robinson,5.10,NaN,0,7576260
5,514,6,1,False,False,3,1,Bowen,3,73,West Ham,Bowen,2,1,2,False,Bowen,7.30,7.50,0,7576260
6,366,7,1,False,False,3,1,B.Fernandes,3,83,Man Utd,B.Fernandes,3,1,3,False,B.Fernandes,8.30,8.40,0,7576260
7,182,8,1,False,False,3,1,Palmer,3,111,Chelsea,Palmer,2,1,2,False,Palmer,11.10,NaN,0,7576260
8,328,9,1,False,False,3,1,M.Salah,3,137,Liverpool,M.Salah,14,1,14,False,M.Salah,13.70,NaN,0,7576260
9,351,10,2,True,False,4,1,Haaland,4,147,Man City,Haaland,7,2,14,False,Haaland,14.70,14.80,0,7576260


In [ ]:
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Remove width restriction
pd.set_option("display.colheader_justify", "left")  # Adjust column alignment
pd.set_option("display.float_format", "{:.2f}".format)

In [ ]:
file_path = "FPL_Team_Analysis.csv"
final_df.to_csv(file_path, index=False)

print(f"Data saved to {file_path}")

Data saved to FPL_Team_Analysis.csv
